- En este ejercicio se quiere predecir los precios de inmuebles (variable ```Sale_Price```)

# Ejercicio 1

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score

import warnings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (12, 6)
plt.style.use('seaborn')

df = pd.read_csv('ames_housing.csv').drop(columns='Unnamed: 0')

# Ejercicio 2

In [30]:
def obtener_categorias_df_a_one_hot_encoding(df):
    tmp = df.copy()
    for columna, tipo_columna in tmp.dtypes.items():
        if tipo_columna == 'object':
            leave_one_out_encoding = pd.get_dummies(tmp[columna], drop_first=True, prefix=columna)
            tmp = tmp.drop(columns=columna)
            tmp = pd.concat([tmp, leave_one_out_encoding], axis=1)

    return tmp

In [31]:
codificada = obtener_categorias_df_a_one_hot_encoding(df)

# Ejercicio 3

In [38]:
X = codificada.drop(columns=['Sale_Price'])
y = codificada['Sale_Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

arbol = DecisionTreeRegressor()
arbol.fit(X_train, y_train)

yhat = arbol.predict(X_test)

mae = mean_absolute_error(y_test, yhat)
r2 = r2_score(y_test, yhat)

print(f'El MAE del modelo fue: {mae}\n'
      f'El R2 del modelo fue: {r2}')

El MAE del modelo fue: 26154.769389865563
El R2 del modelo fue: 0.7737355142148538


In [41]:
y.describe()

count      2930.000000
mean     180796.060068
std       79886.692357
min       12789.000000
25%      129500.000000
50%      160000.000000
75%      213500.000000
max      755000.000000
Name: Sale_Price, dtype: float64

- Segun la escala de datos, se puede ver que el modelo tiene un mean_squared_error menor a la desviacion estandar del vector objetivo.
- El modelo explica en un 77.4% la varianza del vector objetivo (R2).

Por lo tanto, el modelo presenta un desempeno aceptable.